In [21]:
import os
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import time
from torch.optim import lr_scheduler



from torch.utils.data import Dataset
from torch.utils.data import random_split

from torchvision import transforms, models
from MyEDFImports import load_all_labels
from tempfile import TemporaryDirectory

In [2]:
data_dir = 'images_(19248, 224, 224)_wav_morlet_sqpy.npy'
data_np = np.load(data_dir)
targets = load_all_labels()

In [3]:
class DatasetFromNp(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = target
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        # Extending 1channel image to be put to three channels
        x = torch.from_numpy(x)
        x = torch.unsqueeze(x, 0)
        x = x.expand(3, -1, -1)
        y = self.target[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.target)


In [10]:
# setting up tranforms for the images (just normalizing pretty much)
data_transforms = transforms.Compose([
    # to tensor can probably be just torch.from_numpy
    # transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

dataset_all = DatasetFromNp(data_np, target=targets, transform=data_transforms)

generator = torch.Generator()  # .manual_seed(22)
train_data, test_data = random_split(dataset_all, [0.8, 0.2], generator=generator)
datasets = {'train': train_data, 'val': test_data}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in
               ['train', 'val']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}
# remove a fixed generator for training


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
# this imshow should be fixed doesn't work yet
def imshow(inp, title=None):
    # inp = inp.numpy().transpose((1, 2, 0))
    # mean = np.array([0.485, 0.456, 0.406])
    # std = np.array([0.229, 0.224, 0.225])
    # inp = std * inp + mean
    # inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)


# not necessarily test_data subset has to be there can be anything else
inputs, classes = next(iter(dataloaders['train']))
grid = torchvision.utils.make_grid(inputs)
# imshow(grid, title= classes)


In [19]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch + 1}/{num_epochs}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()  # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [14]:
model_152 = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
num_ftrs = model_152.fc.in_features
num_ftrs

2048

In [17]:
model_152.fc = nn.Linear(num_ftrs,2)
model_152.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_152 = optim.SGD(model_152.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_152, step_size=7, gamma=0.1)

In [22]:
model_152 = train_model(model_152, criterion, optimizer_152, exp_lr_scheduler,
                       num_epochs=25)

Epoch 1/25
----------


RuntimeError: Input type (torch.cuda.DoubleTensor) and weight type (torch.cuda.FloatTensor) should be the same